## Definitions

In [12]:
import pandas as pd

inputDataset="5dd481247643d218a8dc150a" # Adobe Analytics: Demo Environment postValues
outputDataset="5dd39447c54ff018a8abbc1a" # Recommendations Input Dataset

item_id = "_experience.analytics.customDimensions.eVars.eVar13"
interactionType = "_experience.analytics.customDimensions.eVars.eVar15"
user_id = "_experience.analytics.customDimensions.eVars.eVar1"
brand_name = "_experience.analytics.customDimensions.eVars.eVar16"
timestamp = "timestamp"
tenant_id = "_ibmnaamericaspartnersandbox"

client_context = PLATFORM_SDK_CLIENT_CONTEXT

## Load the purchases data

In [13]:
from platform_sdk.dataset_reader import DatasetReader

dataset_reader = DatasetReader(client_context, inputDataset)
df = dataset_reader.limit(50000).read()
df.head()

,timestamp,_id,productListItems,commerce.order.currencyCode,commerce.productViews.value,receivedTimestamp,endUserIDs._experience.aaid.id,endUserIDs._experience.aaid.namespace.code,endUserIDs._experience.aaid.primary,endUserIDs._experience.mcid.id,...,_experience.analytics.customDimensions.eVars.eVar12,_experience.analytics.customDimensions.eVars.eVar13,_experience.analytics.customDimensions.eVars.eVar14,_experience.analytics.customDimensions.eVars.eVar15,_experience.analytics.environment.browserID,_experience.analytics.environment.operatingSystemID,device.screenHeight,device.screenWidth,device.colorDepth,search.isPaid
0,2019-11-20T08:43:57,2EEA7E6685158000-4015E9C8E9AE846E,[],EUR,None,2019-11-20T08:43:57,5EAE7E69-70E8EAC1,AAID,True,None,...,None,None,None,None,-685160725,-824629874,0,0,0,False
1,2019-11-20T08:43:06,2EEA7E4D05158000-40114AE549B0170B,[],EUR,None,2019-11-20T08:43:06,2A1C965EFD8CD33F-642EF12F3E1B51F4,AAID,False,91635799509879697768509199328383330031,...,None,None,None,None,-685160725,-824629874,1200,1920,24,False
2,2019-11-20T08:43:08,2EEA7E4E05158000-401A46B7A9AE61C0,[],EUR,None,2019-11-20T08:43:08,2A1C965EFD8CD33F-642EF12F3E1B51F4,AAID,False,91635799509879697768509199328383330031,...,None,None,None,None,-685160725,-824629874,1200,1920,24,False
3,2019-11-20T08:43:57,2EEA7E6685158000-40104B67A9AE8EFA,[],EUR,None,2019-11-20T08:43:57,2A1C965EFD8CD33F-642EF12F3E1B51F4,AAID,False,91635799509879697768509199328383330031,...,None,None,None,None,-685160725,-824629874,1200,1920,24,False
4,2019-11-20T08:44:04,2EEA7E6A05158000-4007EACD69AFDBF2,[],EUR,None,2019-11-20T08:44:04,2A1C965EFD8CD33F-642EF12F3E1B51F4,AAID,False,91635799509879697768509199328383330031,...,None,None,None,None,-685160725,-824629874,1200,1920,24,False


## Filtering

In [14]:
# drop nulls
df = df.dropna(subset=[user_id, item_id, interactionType, brand_name])

# only focus on one brand
df = df[df[brand_name] == "Luma Retail"]

## Split items into individual records

In [15]:
# vectorized (no loops) solution for splitting in pandas
# source: https://stackoverflow.com/a/48120674
def split_df(dataframe, col_name, sep):
    orig_col_index = dataframe.columns.tolist().index(col_name)
    orig_index_name = dataframe.index.name
    orig_columns = dataframe.columns
    dataframe = dataframe.reset_index()
    index_col_name = (set(dataframe.columns) - set(orig_columns)).pop()
    df_split = pd.DataFrame(
        pd.DataFrame(dataframe[col_name].str.split(sep).tolist())
        .stack().reset_index(level=1, drop=1), columns=[col_name])
    df = dataframe.drop(col_name, axis=1)
    df = pd.merge(df, df_split, left_index=True, right_index=True, how='inner')
    df = df.set_index(index_col_name)
    df.index.name = orig_index_name

    return df

df2 = split_df(df, item_id, "\|\|")

# Data Prep for saving back to platform

In [16]:
filtered_column_list = [item_id, user_id, interactionType, brand_name, timestamp]

df2 = df2[filtered_column_list]

df2.rename(columns={
    item_id: tenant_id + ".itemId",
    user_id: tenant_id + ".userId",
    interactionType: tenant_id + ".interactionType",
    brand_name: tenant_id + ".brandName"
}, inplace=True)

# Write new dataframe to platform

In [17]:
df2.head()

,_ibmnaamericaspartnersandbox.itemId,_ibmnaamericaspartnersandbox.userId,_ibmnaamericaspartnersandbox.interactionType,_ibmnaamericaspartnersandbox.brandName,timestamp
9,Nadia Elements Shell,91635799509879697768509199328383330031,productView,Luma Retail,2019-11-20T08:44:37
10,Nadia Elements Shell,91635799509879697768509199328383330031,productView,Luma Retail,2019-11-20T08:44:44
11,Nadia Elements Shell,91635799509879697768509199328383330031,productView,Luma Retail,2019-11-20T08:44:59
12,Nadia Elements Shell,91635799509879697768509199328383330031,productView,Luma Retail,2019-11-20T08:44:59
13,Nadia Elements Shell,91635799509879697768509199328383330031,productView,Luma Retail,2019-11-20T08:45:31


In [19]:
df2['timestamp'] = pd.to_datetime(df2['timestamp']).apply(lambda x: x.isoformat())

from data_access_sdk_python.writer import DataSetWriter
writer = DataSetWriter()
writer.write(data_set_id=outputDataset, dataframe=df2, ims_org="91D55B255640B3AE7F000101@AdobeOrg", file_format="json")

INFO:DataAccessSDK:data_set_id 5dd39447c54ff018a8abbc1a, batch_id b26690c0-0b8a-11ea-b5ba-799aacf714bd, view_id 5dd39447c54ff018a8abbc1b
INFO:DataAccessSDK:Current user_id: 2A76F6555697C3AF7F000101@AdobeID
INFO:azure.datalake.store.core:closing stream
INFO:azure.datalake.store.transfer:Transferred tempFile.parquet -> /foundation/data/stage/users/2A76F6555697C3AF7F000101@AdobeID/b26690c0-0b8a-11ea-b5ba-799aacf714bd/5dd39447c54ff018a8abbc1b/1574250110069.json


('5dd39447c54ff018a8abbc1a', 'b26690c0-0b8a-11ea-b5ba-799aacf714bd')